### 载入所需的库
我们依旧会用gensim去做word2vec的处理，会用sklearn当中的SVM进行建模

In [48]:
# -*- coding: utf-8 -*-
"""
Created on 2015.12.09

@author: Hanxiaoyang
"""
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec
import numpy as np
import pandas as pd
import jieba
from sklearn.externals import joblib
from sklearn.svm import SVC
import sys  
import os
# reload(sys)  
# sys.setdefaultencoding('utf8')


os.mkdir("svm_data/svm_model")


### 载入数据，做预处理(分词)，切分训练集与测试集

In [29]:
def load_file_and_preprocessing():
    neg=pd.read_excel('data/neg.xls',header=None,index=None)
    pos=pd.read_excel('data/pos.xls',header=None,index=None)
    
    # jieba 库 对句子分词
    # 这里是创建匿名函数并把函数地址赋值给 cw
    cw = lambda x: list(jieba.cut(x))
    pos['words'] = pos[0].apply(cw)
    neg['words'] = neg[0].apply(cw)
    
    #print pos['words']
    #use 1 for positive sentiment, 0 for negative
    # 自行健 label ，pos = 1，neg = 0
    # concatenate 是组合，np.ones + np.zeros
    y = np.concatenate((np.ones(len(pos)), np.zeros(len(neg))))

    x_train, x_test, y_train, y_test = train_test_split(np.concatenate((pos['words'], neg['words'])), y, test_size=0.2)
    
#     np.save('svm_data/y_train.npy',y_train)
#     np.save('svm_data/y_test.npy',y_test)
    return x_train, x_test, y_train, y_test

In [30]:
# 拆分好测试和验证集
x_train, x_test, y_train, y_test = load_file_and_preprocessing()
print("数据集查看")
print(x_train[5],y_train[5])
print(x_train[6],y_train[5])

数据集查看
['来电', '的', '反应', '很', '慢', '，', '都', '要', '等', '好长时间', '才', '通', '，', '怀疑', '它', '信号', '不好', '，', '果然', '，', '它', '的', '信号', '在', '翻盖', '，', '合盖', '或者', '刚', '挂断', '电话', '时', '居然', '经常', '从', '五格', '变到', '两到', '三格', '，', '有时', '甚至', '变为', '一', '格', '，', '而且', '对', '来电', '反应', '很', '慢', '，', '很', '长时间', '才能', '通', '。'] 0.0
['本书', '一', '开始', '附带', '了', '大量', '的', '历史', '知识', '，', '看', '的', '我', '很', '不耐烦', '，', '觉得', '也', '没', '组织', '好', '头绪', '，', '很乱', '的', '感觉', '，', '但是', '似乎', '大家', '对', '这', '本书', '评价', '还', '不错', '，', '那么', '耐着', '性子', '看', '下去', '，', '从', '第三本', '开始', '，', '真的', '发现', '渐入佳境', '，', '作者', '开始', '找到', '感觉', 'le', '事情', '的', '进展', '开始', '有', '那么些', '意思', '了', '，', '最后', '一口气', '看', '完全', '部五本', '，', '发现', '喜欢', '上', '了', '这', '套书', '，', '看来', '还要', '继续', '追下去', '，', '只是', '可惜', '还', '没出', '完', '，', '真', '希望', '看到', '后面', '，', '不', '知道', '作者', '会', '给', '我们', '怎样', '的', '结果', '？'] 0.0


### 对每个句子的所有词向量取均值，来生成一个句子的vector

In [35]:
def build_sentence_vector(text, size,imdb_w2v):
    """
    1、将每个 word 在 word2vec 模型中映射成为300维向量
    2、句子的向量 = 每个word 的向量平均值
    """
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += imdb_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

### 计算词向量

In [46]:
# 训练 word2vec 模型并生成词向量
def get_train_vecs(x_train,x_test):
    n_dim = 300
    #初始化模型和词表
    imdb_w2v = Word2Vec(size=n_dim, min_count=10)
    imdb_w2v.build_vocab(x_train)
    
    #在评论训练集上建模(可能会花费几分钟)
    imdb_w2v.train(x_train,total_examples=imdb_w2v.corpus_count, epochs=20)
    
    train_vecs = np.concatenate([build_sentence_vector(z, n_dim,imdb_w2v) for z in x_train])
    #train_vecs = scale(train_vecs)
    
#     np.save('svm_data/train_vecs.npy',train_vecs)
    print(train_vecs.shape)
    #在测试集上训练
    imdb_w2v.train(x_test,total_examples=imdb_w2v.corpus_count, epochs=20)
    imdb_w2v.save('svm_data/w2v_model/w2v_model.pkl')
    #Build test tweet vectors then scale
    test_vecs = np.concatenate([build_sentence_vector(z, n_dim,imdb_w2v) for z in x_test])
    #test_vecs = scale(test_vecs)
#     np.save('svm_data/test_vecs.npy',test_vecs)
    print(test_vecs.shape)
    return train_vecs,test_vecs

In [47]:
train_vecs,test_vecs = get_train_vecs(x_train,x_test)

d:\program\python36\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


(16884, 300)


d:\program\python36\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


(4221, 300)


In [39]:
def get_data():
    return train_vecs,y_train,test_vecs,y_test

### 训练svm模型

In [51]:
def svm_train(train_vecs,y_train,test_vecs,y_test):
    clf=SVC(kernel='rbf',verbose=True)
    clf.fit(train_vecs,y_train)
    joblib.dump(clf, 'svm_data/svm_model/model.pkl')
    return clf.score(test_vecs,y_test)

In [52]:
# 计算模型性能
score = svm_train(train_vecs,y_train,test_vecs,y_test)
print("得分：",score)

d:\program\python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[LibSVM]得分： 0.8654347311063729


### 构建待预测句子的向量

In [53]:
def get_predict_vecs(words):
    n_dim = 300
    imdb_w2v = Word2Vec.load('svm_data/w2v_model/w2v_model.pkl')
    #imdb_w2v.train(words)
    train_vecs = build_sentence_vector(words, n_dim,imdb_w2v)
    #print train_vecs.shape
    return train_vecs

### 对单个句子进行情感判断 

In [55]:
def svm_predict(string):
    words=jieba.lcut(string)
    words_vecs=get_predict_vecs(words)
    clf=joblib.load('svm_data/svm_model/model.pkl')
     
    result=clf.predict(words_vecs)
    
    if int(result[0])==1:
        print(string,' positive')
    else:
        print (string,' negative')

In [58]:
##对输入句子情感进行判断
string='电池充完了电连手机都打不开.简直烂的要命.真是金玉其外,败絮其中!连5号电池都不如'
string='书脏了'    
svm_predict(string)

书脏了  negative


d:\program\python36\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
